In [2]:
# Backtesting OVER 500 STOCKS with a Trading Strategy using Object Oriented Programming [OOP]
# https://www.youtube.com/watch?v=R0FLsIsR4Ic

In [11]:
import yfinance as yf
# data handling
import pandas as pd
# calculation purporses
import numpy as np
# technical indictor
import ta
# for visualization
import matplotlib.pyplot as plt

In [46]:
tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]['Symbol']

In [30]:
# table of changes in the S&P 500
tickers_chg = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[1]

In [31]:
tickers_chg

Date  Added                                Removed  \
                  Date Ticker                       Security  Ticker   
0        June 21, 2022    KDP               Keurig Dr Pepper  UA/UAA   
1        June 21, 2022     ON               ON Semiconductor    IPGP   
2         June 8, 2022   VICI                Vici Properties    CERN   
3        April 4, 2022    CPT          Camden Property Trust    PBCT   
4        March 2, 2022    MOH              Molina Healthcare    INFO   
..                 ...    ...                            ...     ...   
301       June 9, 1999    WLP                      Wellpoint     HFI   
302  December 11, 1998    FSR                        Firstar     LDW   
303  December 11, 1998    CCL                 Carnival Corp.     GRE   
304  December 11, 1998   CPWR                      Compuware     SUN   
305      June 17, 1997    CCI  Countrywide Credit Industries     USL   

                                \
                      Security   
0                 Under Armour   
1                IPG Photonics   
2                       Cerner   
3    People's United Financial   
4                   IHS Markit   
..                         ...   
301   Harnischfeger Industries   
302                      Amoco   
303                 General Re   
304                 SunAmerica   
305                     USLife   

                                                Reason  
                                                Reason  
0                     Market capitalization change.[4]  
1                     Market capitalization change.[4]  
2    S&P 500 constituent Oracle Corp. acquired Cern...  
3    S&P 500 constituent M&T Bank Corp. acquired Pe...  
4    S&P 500 constituent S&P Global Inc. acquired I...  
..                                                 ...  
301           Harnischfeger filed for bankruptcy.[231]  
302            British Petroleum purchased Amoco.[232]  
303      Berkshire Hathaway purchased General Re.[232]  
304                     AIG purchased SunAmerica.[232]  
305                          AIG acquired USLife.[233]  

[306 rows x 6 columns]

In [36]:
added = tickers_chg[pd.to_datetime(tickers_chg.Date.Date) >= '2019-01-01'].Added
removed = tickers_chg[pd.to_datetime(tickers_chg.Date.Date) >= '2019-01-01'].Removed

In [45]:
# takes out tickers that are in the "added" list
tickers = tickers[~(tickers.isin(added.Ticker))]

In [68]:
# append removed tickers
tickers = tickers.append(removed.Ticker)

,Ticker,Security
0,UA/UAA,Under Armour
1,IPGP,IPG Photonics
2,CERN,Cerner
3,PBCT,People's United Financial
4,INFO,IHS Markit
...,...,...
64,BHF,Brighthouse Financial
65,GT,The Goodyear Tire & Rubber Company
66,NFX,Newfield Exploration
67,PCG,Pacific Gas & Electric Company


In [70]:
tickers.dropna(inplace=True)
tickers.drop_duplicates(inplace=True)
tickers = tickers.reset_index(drop=True)

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [13]:
class Backtest:
    def __init__(self, symbol):
        self.symbol = symbol
        self.df = yf.download(self.symbol, start='2019-01-01')
        
        if self.df.empty:
            print("No Data Pulled")
        else:
            self.calc_indicators()
            self.generate_signals()
            self.loop_it()
            self.profit = self.calc_profit()
            self.max_dd = self.profit.min()
            self.cumul_profit = (self.profit + 1).prod() - 1
            
    def calc_indicators(self):
        self.df['ma_20'] = self.df.Close.rolling(20).mean()
        # rolling standard deviation
        self.df['vol'] = self.df.Close.rolling(20).std()
        self.df['upper_bb'] = self.df.ma_20 + (2*self.df.vol)
        self.df['lower_bb'] = self.df.ma_20 - (2*self.df.vol)
        self.df['rsi'] = ta.momentum.rsi(self.df.Close, window=6)
        self.df.dropna(inplace=True)
        
    def generate_signals(self):
        conditions = [(self.df.rsi < 30) & (self.df.Close < self.df.lower_bb),
              (self.df.rsi > 70) & (self.df.Close < self.df.upper_bb)]
        choices = ['Buy', 'Sell']
        self.df['signal'] = np.select(conditions, choices)
        # close row before == shifted_close --> makes it more convenient to access signal when Backtester is iterating
        self.df['shifted_Close'] = self.df.Close.shift()
        self.df.dropna(inplace=True)
        
    def loop_it(self):
        position = False
        buydates, selldates = [], []
        
        for index, row in self.df.iterrows():
            if not position and row['signal'] == 'Buy':
                buydates.append(index)
                position = True
            if position:
                if row['signal'] == 'Sell': # or row.shifted_Close < 0.95 * buyprices[-1]:
                    selldates.append(index)
                    position = False
                    
        self.buy_arr = self.df.loc[buydates].Open
        self.sell_arr = self.df.loc[selldates].Open

    def calc_profit(self):
        # gets rid of last buy order if it hasn't triggered and sold yet
        if self.buy_arr.index[-1] > self.sell_arr.index[-1]:
            self.buy_arr = self.buy_arr[:-1]
        
        return (self.sell_arr.values - self.buy_arr.values)/self.buy_arr.values
    
    def chart(self):
        plt.figure(figsize=(20,10))
        plt.plot(self.df.Close)
        plt.scatter(self.buy_arr.index, self.buy_arr.values, marker="^", c='g')
        plt.scatter(self.sell_arr.index, self.sell_arr.values, marker="v", c='r')        

In [71]:
instances = []

for ticker in tickers:
    instances.append(Backtest(ticker))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [72]:
profits , comp_name = [], []

for obj in instances:
    if not obj.df.empty:
        profits.append(obj.cumul_profit)
        comp_name.append(obj.symbol)